In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import sklearn

In [ ]:
epd_df = pd.read_csv("EPD.csv", encoding='utf-8')
epd_df.rename(columns={"Open":"Open_EPD", "Close": "Close_EPD"}, inplace = True)
epd_df["Monthly_Change_EPD"] = epd_df["Open_EPD"] - epd_df["Close_EPD"]

kmi_df = pd.read_csv("KMI.csv", encoding='utf-8')
kmi_df.rename(columns={"Open":"Open_KMI", "Close": "Close_KMI"}, inplace = True)
kmi_df["Monthly_Change_KMI"] = kmi_df["Open_KMI"] - kmi_df["Close_KMI"]
kmi_df = kmi_df.drop(columns=["Date"])

mmp_df = pd.read_csv("MMP.csv", encoding='utf-8')
mmp_df.rename(columns={"Open":"Open_MMP", "Close": "Close_MMP"}, inplace = True)
mmp_df["Monthly_Change_MMP"] = mmp_df["Open_MMP"] - mmp_df["Close_MMP"]
mmp_df = mmp_df.drop(columns=["Date"])

mplx_df = pd.read_csv("MPLX.csv", encoding='utf-8')
mplx_df.rename(columns={"Open":"Open_MPLX", "Close": "Close_MPLX"}, inplace = True)
mplx_df["Monthly_Change_MPLX"] = mplx_df["Open_MPLX"] - mplx_df["Close_MPLX"]
mplx_df = mplx_df.drop(columns=["Date"])

et_df = pd.read_csv("ET.csv", encoding='utf-8')
et_df.rename(columns={"Open":"Open_ET", "Close": "Close_ET"}, inplace = True)
et_df["Monthly_Change_ET"] = et_df["Open_ET"] - et_df["Close_ET"]
et_df = et_df.drop(columns=["Date"])

xom_df = pd.read_csv("XOM.csv", encoding='utf-8')
xom_df.rename(columns={"Open":"Open_XOM", "Close": "Close_XOM"}, inplace = True)
xom_df["Monthly_Change_XOM"] = xom_df["Open_XOM"] - xom_df["Close_XOM"]
xom_df = xom_df.drop(columns=["Date"])

unemployment_df = pd.read_csv("USUnemployment.csv", encoding = 'utf-8')

cci_df = pd.read_csv("CCI.csv", encoding='utf-8')

nasdaq_df = pd.read_csv("IXIC.csv", encoding='utf-8')
nasdaq_df.rename(columns={"Open":"Open_NASDAQ", "Close": "Close_NASDAQ"}, inplace = True)
nasdaq_df["Monthly_Change_NASDAQ"] = nasdaq_df["Open_NASDAQ"] - nasdaq_df["Close_NASDAQ"]
nasdaq_df = nasdaq_df.drop(columns=["Date"])


In [ ]:
result = pd.concat([epd_df, kmi_df, mmp_df, mplx_df, nasdaq_df, et_df, xom_df], axis=1, join="outer")

In [ ]:
result = result.dropna()
result = result.drop(columns=["High", "Low", "Adj Close", "Volume"])
result = result.iloc[1:]
final_result = result.iloc[:-1]

In [ ]:
final_result

In [ ]:
unemployment_monthly = pd.DataFrame
unemployment_monthly_dates = []
unemployment_monthly_rates = []
for x in range(64,72):
    unemployment_monthly_rates.append(unemployment_df["Jan"][x])
    unemployment_monthly_rates.append(unemployment_df["Feb"][x])
    unemployment_monthly_rates.append(unemployment_df["Mar"][x])
    unemployment_monthly_rates.append(unemployment_df["Apr"][x])
    unemployment_monthly_rates.append(unemployment_df["May"][x])
    unemployment_monthly_rates.append(unemployment_df["Jun"][x])
    unemployment_monthly_rates.append(unemployment_df["Jul"][x])
    unemployment_monthly_rates.append(unemployment_df["Aug"][x])
    unemployment_monthly_rates.append(unemployment_df["Sep"][x])
    unemployment_monthly_rates.append(unemployment_df["Oct"][x])
    unemployment_monthly_rates.append(unemployment_df["Nov"][x])
    unemployment_monthly_rates.append(unemployment_df["Dec"][x])

unemployment_monthly_rates.append(3.6)
unemployment_monthly_rates.append(3.5)
unemployment_monthly_rates.append(4.4)
unemployment_monthly_rates.append(14.7)
unemployment_monthly_rates.append(13.3)
unemployment_monthly_rates.append(11.1)
unemployment_monthly_rates.append(10.2)
unemployment_monthly_rates.append(8.4)
unemployment_monthly_rates.append(7.8)
unemployment_monthly_rates.append(6.9)
unemployment_monthly_rates.append(6.7)
unemployment_monthly_rates.append(6.7)

del unemployment_monthly_rates[0:11]

In [ ]:
final_result["Unemployment_Rate"] = unemployment_monthly_rates

In [ ]:
cci_df_new = cci_df[cci_df["LOCATION"] == "USA"]

In [ ]:
cci_df_new

In [ ]:
cci_df_new_values = cci_df_new["Value"]
final_result["CCI_Values"] = cci_df_new_values.values

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)

def RFE_feature_selection():
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    select = RFE(DecisionTreeRegressor(random_state=1), n_features_to_select=3)
    
    #fit the RFE selector to the training data
    select.fit(X_train, y_train)

    #transform training and testing sets so only the selected features are retained
    X_train_selected = select.transform(X_train)
    X_test_selected = select.transform(X_test)
    
    model = Ridge().fit(X=X_train_selected, y=y_train)
    
    print("Selected features after RFE:")
    for i in range(len(select.get_support())):
        if select.get_support()[i]:
            print(features.columns[i]) #gets the features for which get_support is true
    
    accuracy_train = model.score(X_train_selected, y_train)
    accuracy_test = model.score(X_test_selected, y_test)

    print("\nkNN Classification performance with selected features:")
    print("\t Prediction accuracy on the train data:", f"{accuracy_train:.2%} \n")
    print("\t Prediction accuracy on the test data:", f"{accuracy_test:.2%} \n")

RFE_feature_selection()

In [ ]:
features = final_result[["Monthly_Change_KMI","Monthly_Change_MPLX", "CCI_Values"]]
target = final_result["Monthly_Change_ET"]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR

estimators = {
    'Linear Regression': LinearRegression(), 
    'Ridge': Ridge(alpha = 0.001),
    'Lasso': Lasso(alpha = 0.001),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(n_neighbors = 3),
    'Linear SVC': LinearSVR(C=0.01, max_iter = 100)
}

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def regressors_percentage_split():
    X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=37)
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    for estimator_name, estimator_object in estimators.items():
        estimator_object.fit(X=X_train_scaled, y=y_train)
        accuracy = estimator_object.score(X_test_scaled, y_test)
        print(estimator_name)
        print("\t Prediction accuracy on the test data:", f"{accuracy:.2%} \n")

In [ ]:
regressors_percentage_split()

In [ ]:
param_grid = {"alpha":[0.001, 0.01, 0.1, 1, 10, 100]}

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#import GridSearchCV and fit GridSearchCV
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(Ridge(), param_grid, cv=5)

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

#fit the grid search object on the training data (CV will be performed on this)
grid_search.fit(X=X_train, y=y_train)

#result of grid search
print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

#the performance of the best found parameters on the test set
#this is what you report for the evaluation of your model
print("Test set score: ", grid_search.score(X_test, y_test))

In [ ]:
model = Lasso(alpha=0.001).fit(X=X_train, y=y_train)

In [ ]:
model.predict([[0.005, -5.123, 98.67]])

In [ ]:
import pickle
with open('model_pickle', 'wb') as f:
    pickle_dump(model, f)
with open('model_pickle', 'rb') as f:
    mp = pickle.load(f)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

et2_df = pd.read_csv("ET2.csv", encoding='utf-8', date_parser = True)

In [ ]:
et_train = et2_df[et2_df['Date'] < '2019-01-01'].copy()

et_test = et2_df[et2_df['Date'] >= '2019-01-01'].copy()

training_data = et_train.drop(['Date', 'Adj Close'], axis=1)

In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)

In [ ]:
X_train = []
y_train = []

for i in range(60, training_data.shape[0]):
    X_train.append(training_data[i-60:i])
    y_train.append([i,0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
past_60_days = et_train.tail(60)
df_past_60 = past_60_days.append(et_test, ignore_index = True)
df_past_60 = df_past_60.drop(["Date", "Adj Close"], axis=1)

inputs = scaler.fit_transform(df_past_60)

In [ ]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])
    
X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = 'relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1, activation = 'linear'))

In [ ]:
opt = SGD(lr = 0.01, momentum = 0.9, clipnorm = 1.0)
regressor.compile(optimizer=opt, loss='mean_squared_logarithmic_error', metrics=["mse"])
regressor.fit(X_train, y_train, epochs=10, batch_size=128)